In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [42]:
!pip install transformers newsapi-python --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Imports

In [43]:
from transformers import pipeline
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import os

API and Summarizer For Real-Time Interaction

In [44]:
NEWS_API_KEY = '' 
#use api token from newsapi.org
newsapi = NewsApiClient(api_key=NEWS_API_KEY)

summarizer = pipeline("summarization", model ="facebook/bart-large-cnn")

Device set to use cpu


Chunking Longer Texts

In [45]:
def chunk_text(text, max_len=1024):
    words = text.split()
    for i in range(0, len(words), max_len):
        yield " ".join(words[i:i+max_len])

summaries = []
for chunk in chunk_text(input_text):
    summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)
    summaries.append(summary[0]['summary_text'])

final_summary = " ".join(summaries)

In [53]:
topic = input("Enter a Topic To Summarize (e.g. : AI, Climate): ")

Enter a Topic To Summarize (e.g. : AI, Climate):  OpenAI


Time Tracker

In [54]:
today = datetime.utcnow()
yesterday = today - timedelta(days=5)

from_str = yesterday.strftime('%Y-%m-%dT%H:%M:%S')
to_str = today.strftime('%Y-%m-%dT%H:%M:%S')
print(topic)

OpenAI


In [55]:
all_articles = newsapi.get_everything(
    q=topic,
    from_param=from_str,
    to=to_str,
    language='en',
    sort_by='relevancy',
    page_size=20
)

In [56]:
contents = []
print(f"Number of articles fetched: {len(all_articles['articles'])}")
for article in all_articles['articles']:
    title = article.get('title', '')
    desc = article.get('description', '')
    if title or desc:
        contents.append(f"{title}. {desc}")

input_text = " ".join(contents)
input_text = input_text[:3000]
print(input_text)

Number of articles fetched: 20
OpenAI awarded $200 million US defense contract. OpenAI is officially on the Pentagon’s payroll. The Department of Defence announced a $200 million contract with OpenAI to provide the US government with new artificial intelligence tools, including those used for proactive cyber defense. In a post outlining … Inside Microsoft’s complicated relationship with OpenAI. Beyond the selfies between Microsoft CEO Satya Nadella and OpenAI CEO Sam Altman, and the friendly conversations between the pair on stage, all is not well with Microsoft's $13 billion AI investment. Over the past year, multiple reports have painted a picture… The ‘OpenAI Files’ will help you understand how Sam Altman’s company works. For about a year, Tyler Johnston has been collecting public information about the inner workings of OpenAI, and for the past month, he’s been working on a report to help the public understand and visualize it.  That report, dubbed The OpenAI Files, is out tod… Open

In [57]:
if input_text:
    summary = summarizer(input_text, max_length=180, min_length=80, do_sample=False)
    print(f"\n📝 Summary for '{topic}':\n{summary}")
else:
    print("No articles found for the given topic.")


📝 Summary for 'OpenAI':
[{'summary_text': 'OpenAI awarded $200 million US defense contract. Google tests real-time AI voice chats in Search. Tech execs from Palantir, Meta, and OpenAI have joined the Army Reserve. Google is building Search Live into AI Mode, allowing you to have a back-and-forth voice conversation with the company’s AI chatbot. The test, which is rolling out now to Labs users in the US, currently doesn’t support camera-shar…'}]
